# Import Data

In [ ]:
import pandas as pd
import numpy as np
import datetime as datetime
from datetime import datetime
from dateutil.parser import parse
import time
from sklearn import preprocessing

In [ ]:
alldata=pd.read_csv("raw_data.txt", sep='|')

# Delete Features and Add Head

In [ ]:
def delete_and_add_head(alldata):
# delete constant feature
    del alldata[16]
    del alldata[48]
    del alldata[60]
    del alldata[62]

    head=['auth_id','acct_id','fraud_ind','acct_actvn_dt','acct_avl_cash_before_amt','acct_avl_money_before_amt','acct_credit_limit','acct_curr_balance','acct_multicard_ind','acct_open_dt','acct_prod_cd','acct_type_cd','address_vfcn_frmt_cd','address_vfcn_respns_cd','apprd_authzn_cnt','apprd_cash_authzn_cnt','authzn_acct_stat_cd','authzn_amount','authzn_type_cd','authzn_char_cd','authzn_opset_id','authzn_orig_src_id','authzn_outstd_amt','authzn_outstd_cash_amt','authzn_rqst_proc_cd','authzn_rqst_proc_dt','authzn_rqst_proc_tm','authzn_rqst_type_cd','authzn_trmnl_pin_capblt_num','avg_dly_authzn_amt','card_vfcn_2_respns_cd','card_vfcn_2_vldtn_dur','card_vfcn_msmt_reas_cd','card_vfcn_presnc_cd','card_vfcn_respns_cd','card_vfcn2_vldtn_cd','cardholder_present_cd','currency_rate','elctr_cmrc_ind_cd','home_phn_num_chng_dur','hotel_stay_car_rentl_dur','last_adr_chng_dur','last_plstc_rqst_reas_cd','mrch_type_cd','mrch_country_cd','new_user_added_dur','phn_chng_snc_appn_ind','pin_vldtn_ind','plstc_actvn_dt','plstc_actvn_reqd_ind','plstc_frst_use_ts','plstc_isu_dur','plstc_prev_curr_cd','plstc_rqst_ts','pos_cond_cd','pos_entry_mthd_cd','rcurg_authzn_ind','rvrsl_ind','src_crcy_cd','trmnl_attndnc_cd','trmnl_capblt_cd','trmnl_clasfn_cd','trmnl_id','trmnl_pin_capblt_cd','distance_from_home']
    alldata.columns=head

    #delete non-sense feature
    del alldata['trmnl_id']

    #delete skewed and non significance features
    del alldata['acct_type_cd']
    del alldata['authzn_opset_id']
    del alldata['authzn_rqst_proc_cd']
    del alldata['card_vfcn_msmt_reas_cd']
    del alldata['phn_chng_snc_appn_ind']
    del alldata['arqc_rslt_cd']
    del alldata['pin_blk_cd']
    del alldata['sendr_rsidnl_cntry_cd']
    del alldata['src_crcy_dcml_psn_num']
    del alldata['trmnl_id']

    print alldata.shape

delete_and_add_head(alldata)

# Group Features

In [ ]:
# group merchant type
def transform_merchant_type(x):
    if x<2000:
        x = "con_service"
    elif x>=2000 and x<3000:
        x="wholesale"
    elif x>=3000 and x<=3350:
        x="airline"
    elif x>=3351 and x<=3500:
        x="car_rental"   
    elif x>=3501 and x<=4000:
        x="hotel"
    elif x>=4001 and x<4812:
        x="transportation"
    elif x>=4812 and x<5000:
        x="utilities"
    elif x>=5000 and x<=5199:
        x= "wholesale"
    elif x>=5200 and x<5500:
        x="retail"
    elif x>=5500 and x<5600:
        x="automobiles"
    elif x>=5600 and x<5700:
        x="clothing"
    elif x>=5700 and x<6000:
        x="miscellaneous"
    elif x>=6000 and x<7210:
        x= "service_provider"
    elif x>=7210 and x<7311:
        x="personal_service"
    elif x>=7311 and x<7531:
        x="business"
    elif x>=7531 and x<7829:
        x="repair"
    elif x>=7829 and x<8000:
        x="amusement"
    elif x>=8000 and x<9000:
        x="professional_service"
    elif x>=9000:
        x="government"
    elif x==9401:
        x="other"
    elif x==7011:
        x="hotel"
    elif x>=4511:
        x="airline"
    elif x==7512:
        x="car_rental"
    elif x in [7375,7379,7829,8734]:
        x="wholesale"
    elif x in [4829,6050,6051,6529,6530,6534]:
        x="quasi_cash"
    
    return x

# group countries
def transform_country(x):    
    if x in [660,28,533,44,52,535,92,136,192,531,212,214,308,312,332,388,474,500,630,652,659,662,663,670,534,780,796,850,84,188,222,320,340,484,558,591,32,68,76,152,170,218,238,254,328,600,604,740,858,862]:
        x="America"
    elif x ==840:
        x="USA"
    elif x in [398,417,762,795,860,51,31,48,196,268,368,376,400,414,422,275,512,634,682,760,792,784,887,96,116,360,418,458,104,608,702,764,626,704,4,50,64,356,364,462,524,586,144,156,344,446,392,408,410,496,158]:
        x="Asia"
    elif x in [40,56,250,276,280,438,442,492,528,756,8,20,70,191,292,300,336,380,807,470,499,620,674,688,705,724,248,830,208,233,234,246,831,352,372,832,428,440,833,578,744,752,826,112,100,203,348,498,616,642,643,703,804]:
        x="Europe"
    else:
        x="Others"
    return x
    
# group currency rate
def transform_currency(x):    
    if x ==0:
        x="a"
    elif x ==1000000:
        x="b"
    else:
        x="c"
    return x

def transform_src(x):    
    if x ==840:
        x="a"
    else:
        x='b'
    return x

In [ ]:
def group_features(df):
    df['mrch_country_cd'] = df.mrch_country_cd.apply(transform_country)
    df['mrch_type_cd'] = df.mrch_type_cd.apply(transform_merchant_type)
    df['currency_rate'] = df.currency_rate.apply(transform_currency)
    #df['acct_prod_cd']=df.acct_prod_cd.apply(transform_acct_prod_cd)
    df['src_crcy_cd'] = df.src_crcy_cd.apply(transform_src)
group_features(alldata)

# Datetime Feature Engineering
## Raw Datatime Features:
#### 1. authzn_rqst_proc_dt: 
#### Authorization date, *no missing
#### 2. authzn_rqst_proc_tm: 
#### Authorization time, *no missing
#### 3. acct_open: 
#### Account open date *no missing
#### 4. acct_actvn_dt: 
#### First plastic on this account was activated, *With Missing. fill na=mean
#### 5. plstc_actvn_dt: 
#### Newly issued plastic was activated, *With Missing. fill na=mean
#### 6.  plstc_frst_use_ts: 
#### Date and time when the plastic was first used, *With Missing. fill na=mean
#### 7.  plstc_rqst_ts:
#### Date and time when newly issued card was requested, *With Missing. fill na=mean

## Extract Features From Raw Datetime Features
### 1. plstc_rqst_ti-plstc_use_ti

In [ ]:
def transform_date(x):
    if (x!='null'):
        x=parse(x)
        x=time.mktime(x.timetuple())
        return x  
    else:
        return 'null'              
        
def days1(x):
    if (x[0]!='null')&(x[1]!='null'):
        if x[0]<x[1]:
            return 'earlier'
        else:
            return 'later'         
    else:
        return 'null'

def date_diff1(alldata):
    alldata['plstc_frst_use_ts'] = alldata.plstc_frst_use_ts.apply(transform_date)
    alldata['plstc_rqst_ts'] = alldata.plstc_rqst_ts.apply(transform_date)
    alldata['tuple']=zip(alldata['plstc_rqst_ts'],alldata['plstc_frst_use_ts'])
    alldata['plstc_rqst_ti-plstc_use_ti']=alldata.tuple.apply(days1)
    print alldata['plstc_rqst_ti-plstc_use_ti'].value_counts()
    del alldata['tuple']
    
date_diff1(alldata)

### 2. 'acct_actv-acct_open', all>0

In [ ]:
def days2(x):
    if (x[0]!='null'):
        date_format = "%Y-%m-%d"
        a = datetime.strptime(x[0], date_format)
        b = datetime.strptime(x[1], date_format)
        delta = a - b
        
        if delta.days>366:
            return '>a_year'
        else:
            return '<a_year'         
    else:
        return 'null'

def date_diff2(alldata):   
    alldata['tuple']=zip(alldata['acct_actvn_dt'],alldata['acct_open_dt'])
    alldata['acct_actv-acct_open']=alldata.tuple.apply(days2)
    print alldata['acct_actv-acct_open'].value_counts()
    del alldata['tuple']
    
date_diff2(alldata)

### 3. Account open date till 2013-10-1

In [ ]:
def length(x):
    date_format = "%Y-%m-%d"
    a = datetime.strptime(x, date_format)
    b = datetime.strptime(('2013-10-1'), date_format)
    delta = b - a
    return delta.days

def open_days(alldata):
    alldata['acct_open_length']=alldata.acct_open_dt.apply(length)

open_days(alldata)

### 4. plstc_actv_dt-acct_open_dt , all>0

In [ ]:
def days3(x):
    if (x[0]!='null'):
        date_format = "%Y-%m-%d"
        a = datetime.strptime(x[0], date_format)
        b = datetime.strptime(x[1], date_format)
        delta = a - b
        
        if delta.days>366:
            return '>a_year'
        else:
            return '<a_year'
    else:
        return 'null'
    
def date_diff3(alldata):
    alldata['tuple']=zip(alldata['plstc_actvn_dt'],alldata['acct_open_dt'])#all "plstc_actv_dt" were later than "acct_actv_dt"
    alldata['plstc_actv_dt-acct_open_dt']=alldata.tuple.apply(days3)
    print alldata['plstc_actv_dt-acct_open_dt'].value_counts()
    del alldata['tuple']

date_diff3(alldata)

###  5. plstc_actv_dt-acct_actv_dt, all>=0, most are on the same day


In [ ]:
def days4(x):
    if (x[0]!='null'):
        date_format = "%Y-%m-%d"
        a = datetime.strptime(x[0], date_format)
        b = datetime.strptime(x[1], date_format)
        delta = a - b
        
        if delta.days<366:
            return '<a_year'
        else:
            return '>a_year' 
    else:
        return 'null'

def date_diff4(alldata):
        alldata['tuple']=zip(alldata['plstc_actvn_dt'],alldata['acct_actvn_dt'])#all "plstc_actv_dt" were later than "acct_actv_dt"
        alldata['plstc_actv_dt-acct_actv_dt']=alldata.tuple.apply(days4)
        print alldata['plstc_actv_dt-acct_actv_dt'].value_counts()
        del alldata['tuple']

date_diff4(alldata)

### 6. Authorization time: Classify as Morining or Evening 

In [ ]:
def morning(x):
    if x>=8 and x<=20:
        x='m'
    else:
        x='n'
    return x
def get_m(alldata):
    alldata['authzn_rqst_proc_tm'] = alldata.authzn_rqst_proc_tm.apply(lambda x: x.rsplit('.', 1)[0])
    dac = np.vstack(alldata.authzn_rqst_proc_tm.apply(lambda x: list(map(int, x.split(':')))).values)
    alldata['authzn_rqst_hour'] = dac[:,0]
    alldata['authzn_rqst_hour']=alldata.authzn_rqst_hour.apply(morning)
    
get_m(alldata)

## Delete raw features

In [ ]:
def delete_time(alldata):
    del alldata['acct_actvn_dt'],alldata['acct_open_dt'],alldata['plstc_actvn_dt']
    del alldata["authzn_rqst_proc_dt"],alldata["authzn_rqst_proc_tm"]
    del alldata['plstc_rqst_ts'],alldata['plstc_frst_use_ts']

delete_time(alldata)

# Categorical--Get Dummy

In [ ]:
def apply_one_hot_encoding(pd_frame, column_list):
    new_column_list = []
    for col in column_list:
        tmp = pd.get_dummies(pd_frame[col], prefix=col)
        new_column_list.append(tmp)
    new_pd_frame = pd.concat(new_column_list+[pd_frame], axis=1)
    new_pd_frame.drop(column_list, inplace=True, axis=1)
    return new_pd_frame
columns=[                  
            "acct_multicard_ind",          
            "address_vfcn_frmt_cd",        
            "address_vfcn_respns_cd",     
            "authzn_acct_stat_cd",         
            "authzn_type_cd",              
            "authzn_char_cd",             
            "authzn_orig_src_id",          
            "authzn_rqst_type_cd",         
            "authzn_trmnl_pin_capblt_num", 
            "card_vfcn_2_respns_cd",    
            'card_vfcn_presnc_cd',    
            "card_vfcn_respns_cd",         
            "card_vfcn2_vldtn_cd",         
            "cardholder_present_cd",
            'currency_rate',
            "elctr_cmrc_ind_cd", 
            "last_plstc_rqst_reas_cd",     
            "mrch_country_cd",             
            "pin_vldtn_ind",               
            "plstc_actvn_reqd_ind",        
            "plstc_prev_curr_cd",          
            "pos_cond_cd",                 
            "pos_entry_mthd_cd",          
            "rcurg_authzn_ind",            
            "rvrsl_ind", 
            'src_crcy_cd',
            "trmnl_attndnc_cd",            
            "trmnl_capblt_cd",             
            "trmnl_clasfn_cd",             
            "trmnl_pin_capblt_cd",         
            "acct_prod_cd",               
            "mrch_type_cd",
            'authzn_rqst_hour',
            'plstc_actv_dt-acct_actv_dt',
            'plstc_actv_dt-acct_open_dt',
            'acct_actv-acct_open',
            'plstc_rqst_ti-plstc_use_ti',
            '7days_4hour',
            '7days_5hour',
            '7days_6hour',
            '15days_4hour',
            '15days_5hour',
            '15days_6hour',
            '30days_4hour',
            '30days_5hour',
            '30days_6hour'
            ]

In [ ]:
allfeatures = apply_one_hot_encoding(alldata, columns)
allfeatures.to_csv('preprocessed_data.txt',sep='|',index=None)